In [16]:
import pandas as pd 

# read df.to_csv('./data/training_data.csv', index=False)

df = pd.read_csv('./data/training_data.csv')
df.head()

,prompt,paragraph,file_name,prompt_length,paragraph_length,length_difference,qa_data,para_data
0,<|im_start|>user\n Who is the author of the ca...,We are pleased that the following section auth...,Physiology_Levy,123,191,68,<|im_start|>system\n You are an AI medical ass...,<|im_start|>system\n You are an AI medical ass...
1,<|im_start|>user\n What is the primary source ...,The human body consists of billions of cells t...,Physiology_Levy,181,251,70,<|im_start|>system\n You are an AI medical ass...,<|im_start|>system\n You are an AI medical ass...
2,<|im_start|>user\n What is the role of the liv...,els can be stored and then mobilized when inge...,Physiology_Levy,203,122,-81,<|im_start|>system\n You are an AI medical ass...,<|im_start|>system\n You are an AI medical ass...
3,<|im_start|>user\n What is the primary functio...,Gastrointestinal tract: Digests and absorbs fu...,Physiology_Levy,156,39,-117,<|im_start|>system\n You are an AI medical ass...,<|im_start|>system\n You are an AI medical ass...
4,<|im_start|>user\n What is the primary functio...,Endocrine system: Maintains the blood levels o...,Physiology_Levy,287,124,-163,<|im_start|>system\n You are an AI medical ass...,<|im_start|>system\n You are an AI medical ass...


In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "ehartford/dolphin-2.2.1-mistral-7b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
#bnb_config = BitsAndBytesConfig(
#    load_in_8bit=True,
#)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    max_length=1024,
    padding="max_length",
    #add_eos_token=True,
    #add_bos_token=True,
    truncation=True,
)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
from datasets import Dataset

# Assuming 'df' is your pandas DataFrame and it has columns 'qa_data' and 'para_data'
# Separate the data into two dictionaries

import random
random.seed(42)

n = 5000
para_data_sel = random.sample(df["para_data"].tolist(), n)

qa_data_dict = {
    "text": list(df["qa_data"]), # Convert the 'qa_data' column to a list
}

para_data_dict = {
    "text": para_data_sel, # Convert the 'para_data' column to a list
}

# Create two separate Hugging Face Dataset objects
qa_dataset = Dataset.from_dict(qa_data_dict)
para_dataset = Dataset.from_dict(para_data_dict)

print(qa_dataset[0])
print(para_dataset[0])

{'text': '<|im_start|>system\n You are an AI medical assistant who will answer questions accurately.<|im_end|>\n<|im_start|>user\n Who is the author of the cardiovascular system section in the seventh edition?<|im_end|>\n<|im_start|>assistant\n The author of the cardiovascular system section in the seventh edition is Dr. Withrow Gil Wier.<|im_end|>\n<|im_start|>user\n Who are the authors of the nervous system section in the seventh edition?<|im_end|>\n<|im_start|>assistant\n The authors of the nervous system section in the seventh edition are Drs. Kalman Rubinson and Eric Lang.<|im_end|>\n<|im_start|>user\n Who are the authors of the respiratory system section in the seventh edition?<|im_end|>\n<|im_start|>assistant\n The authors of the respiratory system section in the seventh edition are Drs. Michelle Cloutier and Roger Thrall.<|im_end|>\n<|im_start|>user\n Who are the authors of the gastrointestinal system section in the seventh edition?<|im_end|>\n<|im_start|>assistant\n The author

In [20]:
def generate_and_tokenize_prompt(prompt):
    return tokenizer(prompt["text"], truncation=True, padding="max_length", max_length=1024)

train_dataset, val_dataset = qa_dataset, para_dataset

tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = val_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/86353 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [21]:
print(tokenized_val_dataset[0])
print(len(tokenized_val_dataset[0]["input_ids"]))

{'text': '<|im_start|>system\n You are an AI medical assistant who will accurately write parts of medical textbooks.<|im_end|>\n<|im_start|>user\n energy expenditure by stimulating physical activity, energy expenditure, and thermogenesis, and these may be the most important catabolic effects mediated by leptin through the hypothalamus. Although the effects of leptin on food intake and energy expenditure can be readily demonstrated in nonobese mice and humans, the anorexigenic response of leptin is blunted in states of obesity despite high levels of circulating leptin. Such leptin resistance in obese mice can be bypassed by intraventricular injection of leptin. In keeping with this observation, injections of leptin in obese humans fail to affect food intake and energy expendit ure, dashing initial enthusiasm of leptin therapy for obesity. In rodents and humans, loss-of-function mutations affecting components of the leptin pathway give rise to massive obesity. Mice with mutations that di

In [22]:
import matplotlib.pyplot as plt

def plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
    lengths += [len(x['input_ids']) for x in tokenized_val_dataset]
    print(len(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=20, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.show()

# plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

In [23]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [24]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
   

In [25]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 680329728 || all params: 4432417280 || trainable%: 15.348954870963773


In [26]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)
model = accelerator.prepare_model(model)

In [27]:
import wandb, os
wandb.login()

wandb_project = "medqa_mixed_test"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [28]:
model.to("cuda")


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=256, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=256, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
              )
              (k_proj): Linear4bit(
                (lora_dropout): ModuleDic

In [29]:
import transformers
from datetime import datetime

project = "medqa_mixed_test"
base_model_name = "mistral_dolphin_2_2_1"
run_name = base_model_name + "-" + project
output_dir = "./train_outputs/" + run_name

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=200,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        num_train_epochs=1,
        learning_rate=1e-6, # Want a small lr for finetuning
        bf16=True,
        optim="paged_adamw_8bit",
        logging_steps=50,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=1000,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=500,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 23.65 GiB total capacity; 20.45 GiB already allocated; 13.69 MiB free; 22.36 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF